In [1]:
import glob
import gzip
from bs4 import BeautifulSoup
import lxml
from collections import defaultdict
from tqdm import tqdm
import requests
import random
import sys
import pathlib

from collections import defaultdict

import json
url = 'http://ai-capo-api-lb/spaCy_ner_predictor?text_sentence='  # Load Balancer
result_path = '/nfs/gns/literature/Santosh_Tirunagari/OTAR_Results/FN_Analysis_Experiments/FullTextApr2020/'

pathlib.Path(result_path).mkdir(parents=True, exist_ok=True)



In [2]:
def getfileblocks(file_path):
    subFileBlocks = []

    with open(file_path, 'r') as fh:
        for line in fh:
            if line.startswith('<!DOCTYPE'):
                subFileBlocks.append(line)
            else:
                subFileBlocks[-1] += line

    return subFileBlocks

In [3]:
def get_sentences(file_soup):
#     dict_sentences = defaultdict(list)
    sentences = []
    for each_sentence in file_soup.find_all('plain'):
        sentences.append(each_sentence.text)
        
    return sentences

In [4]:
def get_tags(sentences_):

    set_GP_tags = set()
    set_DS_tags = set()

    for each_sentence in sentences_:
        # print(url + each_uniprot_sentence)
        r = requests.get(url + each_sentence)
        if r.status_code == 200 and r.json()['status'] == 200:
            for each_ml_annotation in r.json()['annotations']:
                if each_ml_annotation[2] == 'GP':
                    set_GP_tags.add(each_ml_annotation[3])
                elif each_ml_annotation[2] == 'DS':
                    set_DS_tags.add(each_ml_annotation[3])
                
    
    
    return list(set_GP_tags),list(set_DS_tags)

In [5]:
xmls= sorted(glob.glob("/nfs/production/literature/shyama/FullText20.04/Annot*.xml"))
xml_path = xmls[0]
xml_path

'/nfs/production/literature/shyama/FullText20.04/Annot_PMC1240624_PMC1474480.xml'

In [6]:
ss = getfileblocks(xml_path)


In [7]:
from collections import defaultdict

class DeepDict(defaultdict):
    def __call__(self):
        return DeepDict(self.default_factory)

In [8]:
for each_article in tqdm(ss):
#     article_tag_dict ={}
    article_tag_dict = DeepDict(DeepDict(list))
    soup = BeautifulSoup(each_article, 'lxml')

    try:
        PMCID = 'PMC'+soup.find(attrs={'pub-id-type':'pmcid'}).text
    except:
        PMCID = 'NA'

#     article_tag_dict['PMCID'] = PMCID

#     try:
#         PMID = soup.find(attrs={'pub-id-type':'pmid'}).text
#     except:
#         PMID = ''
    
    all_sentences = get_sentences(soup)
    GP_list, DS_list = get_tags(all_sentences)
     
    if GP_list and DS_list: # write only if both exist otherwise No
        article_tag_dict[PMCID]['GP'] = GP_list
        article_tag_dict[PMCID]['DS'] = DS_list

        with open(result_path+'tags_'+xml_path.split('/')[-1][:-3]+'jsonl', 'at') as f:
                    json.dump(article_tag_dict,f) #,indent = 2
                    f.write('\n')


 20%|██        | 2023/10000 [3:36:49<47:41:26, 21.52s/it] 

KeyboardInterrupt: 